In [45]:
pd.set_option('display.max_rows', None)


In [12]:
from pybit.unified_trading import HTTP
import pandas as pd
import time

# Создание сессии
session = HTTP(
    api_key="",
    api_secret=""
)

# Функция для получения исторических данных
def get_all_historical_data(symbol, interval, start_time, end_time, chunk_size=200):
    all_data = []
    current_start = start_time

    while current_start < end_time:
        current_end = min(current_start + chunk_size * interval * 60 * 1000, end_time)  # Учитываем размер интервала и ограничения

        response = session.get_kline(
            category="linear",
            symbol=symbol,
            interval=interval,
            start=current_start,
            end=current_end
        )

        if response['retCode'] == 0:
            all_data.extend(response['result']['list'])
            current_start = current_end + 1  # Переход к следующему интервалу, но без +1
        else:
            print("Ошибка в ответе API:", response['retMsg'])
            break

    return all_data

# Установка начального и конечного времени
symbol = "BTCUSDT"
interval = 1  # 1 минута
server_time = session.get_server_time()
end_time = int(server_time['result']['timeSecond']) * 1000
start_time = end_time - 5000000 * 1000  # Минус 10 часов в миллисекундах

# Получение всех данных
data_list = get_all_historical_data(symbol, interval, start_time, end_time)

# Проверка, удалось ли получить данные
if data_list:
    # Создаем пустой список для хранения всех строк данных
    rows = []
    for item in data_list:
        row = [symbol, 'linear'] + item
        rows.append(row)

    # Преобразуем список строк в DataFrame
    df = pd.DataFrame(rows, columns=["symbol", "category", "startTime", "openPrice", "highPrice", "lowPrice", "closePrice", "volume", "turnover"])

    # Преобразуем столбец 'startTime' в формат datetime
    df['startTime'] = pd.to_datetime(df['startTime'], unit='ms')
    
    df.sort_values(by='startTime', inplace=True)

    # Сбрасываем индекс
    df.reset_index(drop=True, inplace=True)

    # Выводим первые несколько строк, чтобы понять структуру данных
    from IPython.display import display
    display(df)
else:
    print("Не удалось получить данные")


C:\Users\Waple\AppData\Local\Temp\ipykernel_3920\3132103545.py:58: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['startTime'] = pd.to_datetime(df['startTime'], unit='ms')


,symbol,category,startTime,openPrice,highPrice,lowPrice,closePrice,volume,turnover
0,BTCUSDT,linear,2024-07-08 13:26:00,57258,57258,57221.3,57233.2,38.058,2178403.1715
1,BTCUSDT,linear,2024-07-08 13:27:00,57233.2,57260.3,57171.7,57185.3,52.502,3002945.0891
2,BTCUSDT,linear,2024-07-08 13:28:00,57185.3,57225.9,57130.2,57218.3,111.069,6349211.2792
3,BTCUSDT,linear,2024-07-08 13:29:00,57218.3,57229.3,57166.3,57166.4,12.844,734609.8698
4,BTCUSDT,linear,2024-07-08 13:30:00,57166.4,57356,57154.8,57322.8,126.104,7220981.2295
...,...,...,...,...,...,...,...,...,...
83329,BTCUSDT,linear,2024-09-04 10:15:00,56531.1,56539.6,56504.6,56539.5,44.132,2494478.3596
83330,BTCUSDT,linear,2024-09-04 10:16:00,56539.5,56563.8,56529.6,56558,47.566,2689826.3265
83331,BTCUSDT,linear,2024-09-04 10:17:00,56558,56573,56518,56523,124.568,7044877.5427
83332,BTCUSDT,linear,2024-09-04 10:18:00,56523,56528.6,56455,56455.1,48.756,2753826.9479


In [13]:
df.to_csv('./history_btc23232.csv')

In [23]:
import numpy as np
import joblib
import pandas as pd
from pybit.unified_trading import HTTP
import time

# Загрузка данных
df = pd.read_csv('./history.csv')
df['startTime'] = pd.to_datetime(df['startTime'])
df.set_index('startTime', inplace=True)

last_timestamp = df.index[-1]

# Инициализация сессии
session = HTTP(
    demo=True,
    api_key="",
    api_secret=""
)

# Определение временных меток
current_time = int(time.time() * 1000)
chunk_size = 200  # Максимальное количество свечей за один запрос
interval = 1  # Интервал свечей в минутах
symbol = "BTCUSDT"
start_time = int(last_timestamp.timestamp() * 1000)
end_time = current_time

# Функция для получения всех данных
def fetch_all_data(session, symbol, interval, start_time, end_time, chunk_size):
    all_data = []
    current_start = start_time

    while current_start < end_time:
        current_end = min(current_start + chunk_size * interval * 60 * 1000, end_time)  # Ограничиваем диапазон запроса

        response = session.get_kline(
            category="linear",
            symbol=symbol,
            interval=interval,
            start=current_start,
            end=current_end
        )

        if response['retCode'] == 0:
            all_data.extend(response['result']['list'])
            current_start = current_end + 1  # Переход к следующему интервалу
        else:
            print("Ошибка в ответе API:", response['retMsg'])
            break

    return all_data

# Получение всех данных с учетом интервалов
new_data = fetch_all_data(session, symbol, interval, start_time, end_time, chunk_size)

if new_data:
    # Преобразование данных в DataFrame
    new_df = pd.DataFrame(new_data, columns=["startTime", "openPrice", "highPrice", "lowPrice", "closePrice", "volume", "turnover"])
    new_df['startTime'] = pd.to_datetime(new_df['startTime'], unit='ms')
    new_df.set_index('startTime', inplace=True)
    new_df = new_df.astype('float64')

    # Объединение старых и новых данных
    df = pd.concat([df, new_df])
    df.sort_values(by='startTime', inplace=True)

    # Сохранение обновленных данных
    df.to_csv('./updated_history_btc.csv')
else:
    print("Нет новых данных для добавления.")


C:\Users\Waple\AppData\Local\Temp\ipykernel_19628\4050529731.py:60: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  new_df['startTime'] = pd.to_datetime(new_df['startTime'], unit='ms')


In [1]:
import numpy as np
from xgboost import XGBClassifier
import joblib
import pandas as pd
from pybit.unified_trading import HTTP
import time
from IPython.display import display

df = pd.read_csv('./updated_history_btc.csv')
# df=df.drop(['Unnamed: 0','symbol','category'], axis=1)
df['startTime'] = pd.to_datetime(df['startTime'])
df.set_index('startTime', inplace=True)

last_timestamp = df.index[-1]

session = HTTP(
    demo=True,
    api_key="",
    api_secret=""
)

current_time = int(time.time() * 1000)

response = session.get_kline(
    category="linear",
    symbol="BTCUSDT",
    interval=1,
    start_time=int(last_timestamp.timestamp() * 1000),
    end_time=current_time
)

if response['retCode'] == 0:
    new_data = response['result']['list']

    new_df = pd.DataFrame(new_data, columns=["startTime", "openPrice", "highPrice", "lowPrice", "closePrice", "volume", "turnover"])
    new_df['startTime'] = pd.to_datetime(new_df['startTime'], unit='ms')
    new_df.set_index('startTime', inplace=True)
    new_df = new_df.astype('float64')

    df = pd.concat([df, new_df])
    df.sort_values(by='startTime', inplace=True)
    df.to_csv('./updated_history_btc.csv')
else:
    print(f"Ошибка получения данных: {response['retMsg']}")

C:\Users\Waple\AppData\Local\Temp\ipykernel_788\123630219.py:36: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  new_df['startTime'] = pd.to_datetime(new_df['startTime'], unit='ms')


In [2]:
import pandas as pd
import numpy as np
df = pd.read_csv('./updated_history_btc.csv')
df.set_index('startTime', inplace=True)
# df=df.drop(['Unnamed: 0','category'],axis=1)
# df = df.rename(columns={'startTime': 'date'})
df = df[~df.index.duplicated(keep='last')]
df=df.tail(15000)
df.to_csv('./updated_history_btc_work.csv')
df

,openPrice,highPrice,lowPrice,closePrice,volume,turnover
startTime,,,,,,
2024-08-30 07:13:00,59355.1,59364.1,59335.7,59364.0,56.374,3.345779e+06
2024-08-30 07:14:00,59364.0,59422.8,59364.0,59400.5,221.280,1.314173e+07
2024-08-30 07:15:00,59400.5,59400.5,59366.0,59367.8,49.948,2.965802e+06
2024-08-30 07:16:00,59367.8,59400.0,59364.0,59386.0,70.762,4.201520e+06
2024-08-30 07:17:00,59386.0,59407.8,59386.0,59406.5,36.583,2.172883e+06
...,...,...,...,...,...,...
2024-09-09 17:08:00,56577.7,56666.0,56577.7,56663.8,258.623,1.464571e+07
2024-09-09 17:09:00,56668.1,56700.0,56630.4,56696.1,292.312,1.656608e+07
2024-09-09 17:10:00,56696.1,56700.0,56517.3,56517.3,693.877,3.928578e+07


In [5]:
def create_target(df, atr_period=17, atr_threshold=60, price_increase_threshold=130, intervals=[3,4, 5, 6, 7, 8, 9, 10]):
    # Расчет скользящих средних
    df['SMA1'] = df['closePrice'].rolling(window=10, min_periods=1).mean()
    df['SMA2'] = df['closePrice'].rolling(window=60, min_periods=1).mean()

    # Расчет ATR
    df['ATR'] = atr(df, n=atr_period)
    
    # Расчет прироста цен за указанные интервалы
    for interval in intervals:
        df[f'price_increase_{interval}'] = df['closePrice'].shift(-interval) - df['closePrice']
    
    # Создание временного сигнала на основе SMA, ATR и прироста цен
    condition_sma_atr = (df['SMA1'] > df['SMA2']) & (df['ATR'] < atr_threshold)
    condition_price_increase = df[[f'price_increase_{interval}' for interval in intervals]].ge(price_increase_threshold).any(axis=1)
    
    df['signal'] = np.where(condition_price_increase, 1.0, 0.0)
    
    # Удаление временных колонок
    df = df.drop(columns=[f'price_increase_{interval}' for interval in intervals] + ['SMA1', 'SMA2', 'ATR'])
    
    return df

# Применение функции
df = create_target(df)
df = df.drop(columns=['prev_close','tr1','tr2','tr3','tr'])

df['signal'].value_counts()
df

,openPrice,highPrice,lowPrice,closePrice,volume,turnover,signal
startTime,,,,,,,
2024-08-30 07:13:00,59355.1,59364.1,59335.7,59364.0,56.374,3.345779e+06,0.0
2024-08-30 07:14:00,59364.0,59422.8,59364.0,59400.5,221.280,1.314173e+07,0.0
2024-08-30 07:15:00,59400.5,59400.5,59366.0,59367.8,49.948,2.965802e+06,1.0
2024-08-30 07:16:00,59367.8,59400.0,59364.0,59386.0,70.762,4.201520e+06,1.0
2024-08-30 07:17:00,59386.0,59407.8,59386.0,59406.5,36.583,2.172883e+06,1.0
...,...,...,...,...,...,...,...
2024-09-09 17:08:00,56577.7,56666.0,56577.7,56663.8,258.623,1.464571e+07,0.0
2024-09-09 17:09:00,56668.1,56700.0,56630.4,56696.1,292.312,1.656608e+07,0.0
2024-09-09 17:10:00,56696.1,56700.0,56517.3,56517.3,693.877,3.928578e+07,0.0


In [6]:
# def create_target(df, atr_period=17, atr_threshold=50):
#     # Расчет скользящих средних
#     df['SMA1'] = df['closePrice'].rolling(window=10, min_periods=1).mean()
#     df['SMA2'] = df['closePrice'].rolling(window=60, min_periods=1).mean()

#     df['ATR'] = atr(df, n=atr_period)
#     df['atr_dynamic_threshold'] = df['ATR'].rolling(window=50, min_periods=1).mean()
#     df['temp_signal'] = np.where((df['SMA1'] > df['SMA2']) & (df['ATR'] < df['atr_dynamic_threshold']), 1.0, 0.0)
    
# #     df['signal'] = np.where(df['closePrice'].shift(5) > df['closePrice'], 0.0, df['temp_signal'])
# #     df['signal'] = np.where(df['closePrice'].shift(4) > df['closePrice'], 0.0, df['temp_signal'])
# #     df['signal'] = np.where(df['closePrice'].shift(3) > df['closePrice'], 0.0, df['temp_signal'])
# #     df['signal'] = np.where(df['closePrice'].shift(2) > df['closePrice'], 0.0, df['temp_signal'])
# #     df['signal'] = np.where(df['closePrice'].shift(1) > df['closePrice'], 0.0, df['temp_signal'])
    
#     df['avg_price'] = (df['highPrice'] + df['lowPrice']) / 2
# #     df['signal'] = np.where(df['avg_price'].shift(5) > df['avg_price'], 0.0, df['temp_signal'])
# #     df['signal'] = np.where(df['avg_price'].shift(4) > df['avg_price'], 0.0, df['signal'])
# #     df['signal'] = np.where(df['avg_price'].shift(3) > df['avg_price'], 0.0, df['signal'])
# #     df['signal'] = np.where(df['avg_price'].shift(2) > df['avg_price'], 0.0, df['signal'])
# #     df['signal'] = np.where(df['avg_price'].shift(1) > df['avg_price'], 0.0, df['signal'])

#     df['signal'] = np.where(
#     (df['closePrice'].shift(1) > df['closePrice']) & 
#     (df['avg_price'].shift(1) > df['avg_price']), 
#     0.0, 
#     df['temp_signal'])
    
#     return df

# # Применение функции
# df = create_target(df)
# df = df.drop(columns=['SMA1', 'SMA2', 'ATR','tr', 'tr1', 'tr2', 'tr3', 'prev_close','atr_dynamic_threshold','temp_signal','avg_price'])
# df['signal'].value_counts()

In [7]:
# def adjust_signals(df):
#     df['next_signal_1'] = df['signal'].shift(-1)
#     df['next_signal_2'] = df['signal'].shift(-2)
#     df['next_signal_3'] = df['signal'].shift(-3)
#     df['next_signal_4'] = df['signal'].shift(-4)
    
#     df['signal'] = np.where(
#         (df['signal'] == 1) &
#         (df['next_signal_1'] == 0) &
#         (df['next_signal_2'] == 0) &
#         (df['next_signal_3'] == 0) &
#         (df['next_signal_4'] == 0),
#         0,
#         df['signal']
#     )
    
#     df = df.drop(columns=['next_signal_1', 'next_signal_2', 'next_signal_3','next_signal_4'])
    
#     return df

# # Применение функции
# df = adjust_signals(df)
# df['signal'].value_counts()


In [8]:
# import numpy as np

# def create_target(ldf,tr_id=False): # покупка 1, продажа 0
#     ldf['SMA1'] = ldf['closePrice'].rolling(window=10, min_periods=1, center=False).mean() #  short
#     ldf['SMA2'] = ldf['closePrice'].rolling(window=60, min_periods=1, center=False).mean() #  long
#     ldf['signal'] = np.where(ldf['SMA1'] > ldf['SMA2'], 1.0, 0.0) # Create signals
    

# create_target(df)
# df=df.drop(columns=['SMA1','SMA2'])
# df['signal'].value_counts()
# # 30 121
# # 50 и 150 - 5 мин

In [9]:
# import plotly.graph_objects as go
# import pandas as pd

# # Создание графика свечей
# fig = go.Figure(data=[go.Candlestick(x=df.index,
#                                      open=df['openPrice'],
#                                      high=df['highPrice'],
#                                      low=df['lowPrice'],
#                                      close=df['closePrice'])])

# # Добавление линии Close Price
# # fig.add_trace(go.Scatter(x=df.index, y=df['closePrice'], mode='lines', name='Close Price', line=dict(color='blue')))

# # Добавление линий скользящих средних (SMA1 и SMA2)
# # fig.add_trace(go.Scatter(x=df.index, y=df['SMA1'], mode='lines', name='SMA1 (10-дневная)', line=dict(dash='dash', color='orange')))
# # fig.add_trace(go.Scatter(x=df.index, y=df['SMA2'], mode='lines', name='SMA2 (60-дневная)', line=dict(dash='dash', color='purple')))

# # Добавление сигналов покупки и продажи
# buy_signals = df[df['signal'] == 1.0]
# sell_signals = df[df['signal'] == 0.0]

# fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['closePrice'], mode='markers', name='Buy Signal', marker=dict(symbol='triangle-up', color='yellow', size=5)))
# fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['closePrice'], mode='markers', name='Sell Signal', marker=dict(symbol='triangle-down', color='purple', size=5)))

# # Настройки графика
# fig.update_layout(
#     title='График свечей BTC/USDT с линиями SMA и сигналами',
#     xaxis_title='Дата',
#     yaxis_title='Цена',
#     xaxis_rangeslider_visible=True,
#     template='plotly_dark'
# )


# # Отображение графика
# fig.show()

In [10]:
''' Technical Indicators '''

#Calculation of moving average
def adaptive_moving_average(series, n):
    ema_fast = series.ewm(span=2).mean()
    ema_slow = series.ewm(span=n).mean()
    abs_diff = np.abs(ema_fast - ema_slow)
    volatility = abs_diff.rolling(n).mean()
    sc = 2 / (volatility + 1)
    ama = ema_fast + sc * (series - ema_fast)
    return ama

def atr(df, n):
    # Расчет True Range
    df['prev_close'] = df['closePrice'].shift(1)
    df['tr1'] = df['highPrice'] - df['lowPrice']
    df['tr2'] = abs(df['highPrice'] - df['prev_close'])
    df['tr3'] = abs(df['lowPrice'] - df['prev_close'])
    df['tr'] = df[['tr1', 'tr2', 'tr3']].max(axis=1)

    # Расчет ATR
    atr = df['tr'].rolling(window=n, min_periods=1).mean()
    return atr

def ma(df, n):
    return pd.Series(df['closePrice'].rolling(n, min_periods=n).mean(), name='MA_' + str(n))

# exponentially weighted moving average 
def ema(df, n):
    return pd.Series(df['closePrice'].ewm(span=n,min_periods=n).mean(), name='EMA_' + str(n))

#Calculation of price momentum
def mom(df, n):     
    return pd.Series(df.diff(n), name='Momentum_' + str(n))  

# rate of change
def roc(df, n):  # Показывает на сколько процентов цена увеличилась/уменьшилась
    M = df.diff(n - 1) ; N = df.shift(n - 1)  
    return pd.Series(((M / N) * 100), name = 'ROC_' + str(n)) 

# relative strength index
def rsi(df, period): # значение перекупленности или перепроданности от 0 до 100
    delta = df.diff().dropna()
    u = delta * 0; d = u.copy()
    u[delta > 0] = delta[delta > 0]; d[delta < 0] = -delta[delta < 0]
    u[u.index[period-1]] = np.mean( u[:period] ) #first value is sum of avg gains
    u = u.drop(u.index[:(period-1)])
    d[d.index[period-1]] = np.mean( d[:period] ) #first value is sum of avg losses
    d = d.drop(d.index[:(period-1)])
    rs = u.ewm(com=period-1, adjust=False).mean() / d.ewm(com=period-1, adjust=False).mean()
    return 100 - 100 / (1 + rs)

# stochastic oscillators slow & fast
def sto(close, low, high, n,id): 
    stok = ((close - low.rolling(n).min()) / (high.rolling(n).max() - low.rolling(n).min())) * 100
    if(id == 0):
        return stok
    else:
        return stok.rolling(3).mean()
    
def macd(df):
    exp1 = df['closePrice'].ewm(span=12, adjust=False).mean()
    exp2 = df['closePrice'].ewm(span=26, adjust=False).mean()
    macd_line = exp1 - exp2
    signal_line = macd_line.ewm(span=9, adjust=False).mean()
    return macd_line, signal_line    
# %K близко к 0 указывает на то, что текущая цена ближе к минимальной цене за период, 
# что может сигнализировать о перепроданности рынка.
# %K близко к 100 указывает на то, что текущая цена ближе к максимальной цене за период,
# что может сигнализировать о перекупленности рынка.

In [11]:
def tech_indi(ldf,tr_id=True):
#     df = df.drop(columns=['openPrice','highPrice','lowPrice','closePrice','turnover','volume','MA63','EMA10','%K10'])

    ldf['ATR14'] = atr(ldf, 41)
    lst_ATR = ['ATR14']
    
    ldf['AMA'] = adaptive_moving_average(ldf['closePrice'], 10)
    lst_AMA = ['AMA']

    ''' Moving Average '''
    ldf['MA21'] = ma(ldf,10)
    ldf['MA63'] = ma(ldf, 30)
    lst_MA = ['MA21','MA63','MA252']

    ''' Exponentially Weighted Moving Average '''
    ldf['EMA10'] = ema(ldf, 10)
    ldf['EMA30'] = ema(ldf, 30)
    ldf['EMA200'] = ema(ldf, 200)
    lst_EMA = ['EMA10','EMA30','EMA200']

    ''' Momentum '''
    ldf['MOM10'] = mom(ldf['closePrice'], 10)
    ldf['MOM30'] = mom(ldf['closePrice'], 30)
    lst_MOM = ['MOM10','MOM30']

    ''' Relative Strength Index '''
    ldf['RSI10'] = rsi(ldf['closePrice'], 10)
    ldf['RSI30'] = rsi(ldf['closePrice'], 30)
    ldf['RSI200'] = rsi(ldf['closePrice'], 200)
    lst_RSI = ['RSI10','RSI30','RSI200']

    ''' Slow Stochastic Oscillators '''
    ldf['%K10'] = sto(ldf['closePrice'], ldf['lowPrice'], ldf['highPrice'],5,0)
    ldf['%K30'] = sto(ldf['closePrice'], ldf['lowPrice'], ldf['highPrice'],10,0)
    ldf['%K200'] = sto(ldf['closePrice'], ldf['lowPrice'], ldf['highPrice'], 20,0)
    lst_pK = ['%K10','%K30','%K200']

    ''' Fast Stochastic Oscillators '''
    ldf['%D10'] = sto(ldf['closePrice'], ldf['lowPrice'], ldf['highPrice'], 10,1)
    ldf['%D30'] = sto(ldf['closePrice'], ldf['lowPrice'], ldf['highPrice'], 30,1)
    ldf['%D200'] = sto(ldf['closePrice'], ldf['lowPrice'], ldf['highPrice'], 200,1)
    lst_pD = ['%D10','%D30','%D200']
    
    ldf['MACD'], ldf['Signal_Line'] = macd(ldf)


In [12]:
tech_indi(df)

In [13]:
df = df.dropna()

In [14]:
NaN_values = (df.isnull().sum() / len(df) * 100).sort_values(ascending = False)
NaN_values

openPrice      0.0
highPrice      0.0
MACD           0.0
%D200          0.0
%D30           0.0
%D10           0.0
%K200          0.0
%K30           0.0
%K10           0.0
RSI200         0.0
RSI30          0.0
RSI10          0.0
MOM30          0.0
MOM10          0.0
EMA200         0.0
EMA30          0.0
EMA10          0.0
MA63           0.0
MA21           0.0
AMA            0.0
ATR14          0.0
tr             0.0
tr3            0.0
tr2            0.0
tr1            0.0
prev_close     0.0
signal         0.0
turnover       0.0
volume         0.0
closePrice     0.0
lowPrice       0.0
Signal_Line    0.0
dtype: float64

In [15]:
# df = df.drop(columns=['openPrice','highPrice','lowPrice','closePrice','volume','MA63','EMA10','%K10'])
# df = df.drop(columns=['openPrice','highPrice','lowPrice','closePrice','turnover','volume'])

# df = df.drop(columns=['openPrice','highPrice','lowPrice','closePrice','turnover','volume'])

In [16]:
from sklearn.model_selection import train_test_split
# tr_df,eval_df = train_test_split(df,test_size=0.2,shuffle=False)

# y_train = tr_df['signal']
# X_train = tr_df.loc[:, tr_df.columns != 'signal']
# y_eval = eval_df['signal']
# X_eval = eval_df.loc[:, eval_df.columns != 'signal']

y_train = df['signal']
X_train = df.loc[:, df.columns != 'signal']

In [17]:
from xgboost import plot_importance,XGBClassifier,XGBRegressor
from sklearn.metrics import classification_report
import joblib

xgb_model = XGBClassifier()

# Обучение модели на тренировочных данных
xgb_model.fit(X_train, y_train)

# Оценка модели на тестовых данных
# y_pred = xgb_model.predict(X_eval)
# print(classification_report(y_eval, y_pred))

model_filename = 'xgb_modelnew1337.pkl'
joblib.dump(xgb_model, model_filename)
print(f"Модель сохранена в файл: {model_filename}")

Модель сохранена в файл: xgb_modelnew1337.pkl


In [18]:
# import plotly.graph_objects as go
# import pandas as pd

# # Предполагается, что eval_df уже содержит предсказанные сигналы и данные для построения графика
# # Если нет, добавьте их, как в предыдущем примере:
# eval_df['predicted_signal'] = y_pred

# # Создание графика свечей
# fig = go.Figure(data=[go.Candlestick(x=eval_df.index,
#                                      open=eval_df['openPrice'],
#                                      high=eval_df['highPrice'],
#                                      low=eval_df['lowPrice'],
#                                      close=eval_df['closePrice'])])

# # Добавление линии для фактических цен закрытия
# fig.add_trace(go.Scatter(x=eval_df.index, y=eval_df['closePrice'], mode='lines', name='Close Price', line=dict(color='blue')))

# # Добавление сигналов покупки и продажи
# buy_signals = eval_df[eval_df['predicted_signal'] == 1.0]
# sell_signals = eval_df[eval_df['predicted_signal'] == 0.0]

# fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['closePrice'], mode='markers', name='Buy Signal', marker=dict(symbol='triangle-up', color='yellow', size=5)))
# fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['closePrice'], mode='markers', name='Sell Signal', marker=dict(symbol='triangle-down', color='purple', size=5)))

# # Настройки графика
# fig.update_layout(
#     title='Фактические цены закрытия и предсказанные сигналы на покупку/продажу (свечи)',
#     xaxis_title='Дата',
#     yaxis_title='Цена закрытия',
#     legend_title='Легенда',
#     xaxis_rangeslider_visible=True,
#     template='plotly_dark'
# )

# # Отображение графика
# fig.show()


In [19]:
import numpy as np
from xgboost import XGBClassifier
import joblib
import pandas as pd
from pybit.unified_trading import HTTP
import time
from IPython.display import display

df = pd.read_csv('./updated_history_btc_work.csv')
# df=df.drop(['Unnamed: 0','symbol','category'], axis=1)
df['startTime'] = pd.to_datetime(df['startTime'])
df.set_index('startTime', inplace=True)

last_timestamp = df.index[-1]

session = HTTP(
    demo=True,
    api_key="",
    api_secret=""
)

current_time = int(time.time() * 1000)

response = session.get_kline(
    category="linear",
    symbol="BTCUSDT",
    interval=1,
    start_time=int(last_timestamp.timestamp() * 1000),
    end_time=current_time
)

if response['retCode'] == 0:
    new_data = response['result']['list']

    new_df = pd.DataFrame(new_data, columns=["startTime", "openPrice", "highPrice", "lowPrice", "closePrice", "volume", "turnover"])
    new_df['startTime'] = pd.to_datetime(new_df['startTime'], unit='ms')
    new_df.set_index('startTime', inplace=True)
    new_df = new_df.astype('float64')

    df = pd.concat([df, new_df])
    df.sort_values(by='startTime', inplace=True)
    df.to_csv('./updated_history_btc_work.csv')
else:
    print(f"Ошибка получения данных: {response['retMsg']}")

C:\Users\Waple\AppData\Local\Temp\ipykernel_788\2449148299.py:36: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  new_df['startTime'] = pd.to_datetime(new_df['startTime'], unit='ms')


In [20]:


import numpy as np
from xgboost import XGBClassifier
import joblib
import pandas as pd
from pybit.unified_trading import HTTP
import time

session = HTTP(
    api_key="",
    api_secret=""
)

df = pd.read_csv('./updated_history_btc_work.csv')
df.set_index('startTime', inplace=True)
df = df[~df.index.duplicated(keep='last')]
# df = df.drop(columns=['openPrice','highPrice','lowPrice','closePrice','volume','MA63','EMA10','%K10'])

df

,openPrice,highPrice,lowPrice,closePrice,volume,turnover
startTime,,,,,,
2024-08-30 07:13:00,59355.1,59364.1,59335.7,59364.0,56.374,3.345779e+06
2024-08-30 07:14:00,59364.0,59422.8,59364.0,59400.5,221.280,1.314173e+07
2024-08-30 07:15:00,59400.5,59400.5,59366.0,59367.8,49.948,2.965802e+06
2024-08-30 07:16:00,59367.8,59400.0,59364.0,59386.0,70.762,4.201520e+06
2024-08-30 07:17:00,59386.0,59407.8,59386.0,59406.5,36.583,2.172883e+06
...,...,...,...,...,...,...
2024-09-09 17:09:00,56668.1,56700.0,56630.4,56696.1,292.312,1.656608e+07
2024-09-09 17:10:00,56696.1,56700.0,56517.3,56517.3,693.877,3.928578e+07
2024-09-09 17:11:00,56517.3,56616.0,56517.3,56548.2,279.527,1.580936e+07


In [21]:
def place_order_buy(session, symbol, side, order_type, price, qty, take_profit_price, take_profit_price_limit,stopLoss,slLimitPrice):
    order_response = session.place_order(
        category="spot",
        symbol=symbol,
        side=side,
        orderType=order_type,
        marketUnit="baseCoin",
        price=price,
        timeInForce="FOK",
        qty=qty,
        takeProfit=str(take_profit_price),
        tpLimitPrice=str(take_profit_price_limit),
        tpTriggerBy='LastPrice',
        tpOrderType="Limit",
        ##
        stopLoss=str(stopLoss),
        slLimitPrice=str(slLimitPrice),
        slTriggerBy='LastPrice',
        slOrderType="Limit"
        ##
    )
    return order_response

def place_order_sell(session, symbol, side, order_type, price, qty):
    order_response= session.place_order(
            category="spot",
            symbol=symbol,
            side=side,
            orderType=order_type,
            marketUnit="baseCoin",
            price=price,
            timeInForce="FOK",
            qty=qty,
        )
    return order_response

In [ ]:
import numpy as np
from xgboost import XGBClassifier
import joblib
import pandas as pd
from pybit.unified_trading import HTTP
import time
from datetime import datetime, timedelta
import threading
pd.set_option('display.max_columns', None)

session = HTTP(
    demo=True,
    api_key="",
    api_secret=""
)

xgb_model = joblib.load('xgb_modelnew1337.pkl')

symbol = "BTCUSDT"


last_signal = 0
order_id = None
last_time = None

while True:
    try:
        current_time = pd.Timestamp.now()
        if current_time.second == 0:
            prev_minute_start_time = int((datetime.now() - timedelta(minutes=1)).timestamp() * 1000)
            print("Сейчас время:")
            print(time.ctime())
            print("Смотрим на пред. свечу:")
            print(pd.to_datetime(prev_minute_start_time, unit='ms'))

            response = session.get_kline(
                category="linear",
                symbol=symbol,
                interval=1,
                limit=1,
                start=prev_minute_start_time,
                end=prev_minute_start_time + 5000
            )
            print(response)

            if response['retCode'] == 0:
                all_data = response['result']['list']
                rows = [item for item in all_data]

                new_df = pd.DataFrame(rows, columns=["startTime", "openPrice", "highPrice", "lowPrice", "closePrice", "volume", "turnover"])

                new_df['startTime'] = pd.to_datetime(new_df['startTime'], unit='ms')
                new_df.set_index('startTime', inplace=True)
                new_df = new_df.astype('float64')

                if last_time is None or new_df.index[0] > last_time:
                    last_time = new_df.index[0]

                    df_fake0 = df.tail(3000)
                    df_fake1 = pd.concat([df_fake0, new_df])
                    tech_indi(df_fake1)
                    last_row_tail = df_fake1.tail(1)
#                     last_row_tail = last_row_tail.drop(columns=['openPrice','highPrice','lowPrice','closePrice','volume','MA63','EMA10','%K10'])

                    df = pd.concat([df, last_row_tail])

                    print(display(df))
                    signal = xgb_model.predict(last_row_tail)
                    print("предсказывает сигнаалл")
                    print(signal)

                    ticker = session.get_tickers(category="spot", symbol="BTCUSDT")
                    price = float(ticker['result']['list'][0]['lastPrice'])
                    take_profit_price = price + 119
                    take_profit_price_limit = take_profit_price + 1
                    ##
                    stopLoss=price-199
                    slLimitPrice=stopLoss-1
                    ##
                    
                    response = session.get_order_history(
                    category="spot",
                    symbol="BTCUSDT",
                    orderId=order_id)
                    
                    history_order=session.get_order_history(
                    category="spot",
                    limit=1)
                    if history_order['result']['list'][0]['stopOrderType']=='tpslOrder' and history_order['result']['list'][0]['side']=='Sell'and history_order['result']['list'][0]['orderStatus']=='Filled':
                        last_signal = 0
                        ##
                    if history_order['result']['list'][0]['stopOrderType']=='BidirectionalTpslOrder' and history_order['result']['list'][0]['side']=='Sell'and history_order['result']['list'][0]['orderStatus']=='Filled':
                        last_signal = 0    
                        ##

                    if signal == 1 and last_signal == 0:
                        mytime = time.ctime()
                        print(f"Покупка BTC в {mytime}")
                        ##
                        order_response = place_order_buy(session, symbol="BTCUSDT", side="Buy", order_type="Limit", price=price, qty=0.1, take_profit_price=take_profit_price, take_profit_price_limit=take_profit_price_limit,stopLoss=stopLoss,slLimitPrice=slLimitPrice)
                        ##
                        order_id = order_response['result']['orderId']
    
                        while True:
                                try:
                                    response = session.get_order_history(
                                        category="spot",
                                        symbol="BTCUSDT",
                                        orderId=order_id
                                    )

                                    if response['retCode'] == 0:
                                        order_info = response['result']

                                        if order_info['list'][0]['orderStatus'] == "Filled":
                                            print(f"Ордер {order_id} выполнен!")
                                            last_signal = 1
                                            break
                                        elif order_info['list'][0]['orderStatus'] != "Filled":
                                            ticker = session.get_tickers(category="spot", symbol="BTCUSDT")
                                            price=float(ticker['result']['list'][0]['lastPrice'])
                                            price=price
                                            print(f"Новая цена {price}.")
                                            order_response = place_order_buy(session, symbol="BTCUSDT", side="Buy", order_type="Limit", price=price, qty=0.1, take_profit_price=take_profit_price, take_profit_price_limit=take_profit_price_limit,stopLoss=stopLoss,slLimitPrice=slLimitPrice)
                                            order_id = order_response['result']['orderId']

                                            print(f"Ордер был отменен.Создали новый {order_id}")
                                            continue
                                    else:
                                        print(f"Ошибка при проверке статуса ордера: {response['retMsg']}")
                                        break  
                                except Exception as e:
                                    print(f"Ошибка при проверке ордера: {e}")
                                time.sleep(0.1)  

#                     elif signal == 0 and last_signal != 0:
#                         mytime = time.ctime()
#                         print(f"Продажа BTC в {mytime}")
#                         order_response = place_order_sell(session, symbol="BTCUSDT", side="Sell", order_type="Limit", price=price, qty=0.1)
#                         order_id = order_response['result']['orderId']
                        
#                         while True:
#                                 try:
#                                     response = session.get_order_history(
#                                         category="spot",
#                                         symbol="BTCUSDT",
#                                         orderId=order_id
#                                     )

#                                     if response['retCode'] == 0:
#                                         order_info = response['result']

#                                         if order_info['list'][0]['orderStatus'] == "Filled":
#                                             print(f"Ордер {order_id} выполнен!")
#                                             last_signal = 0
#                                             ##
#                                             responseOpenOrders = session.get_open_orders(
#                                             category="spot",
#                                             symbol="BTCUSDT")

#                                             if responseOpenOrders['retCode'] == 0:
#                                                 tp_orders = []
#                                                 for order in responseOpenOrders['result']['list']:
#                                                     if order.get('stopOrderType') == 'tpslOrder' and order.get('side') == 'Sell':
#                                                         tp_orders.append(order)

#                                                 for order in tp_orders:
#                                                     order_id = order['orderId']
#                                                     cancel_response = session.cancel_order(
#                                                         category="spot",
#                                                         symbol="BTCUSDT",
#                                                         orderId=order_id
#                                                     )
#                                                     if cancel_response['retCode'] == 0:
#                                                         print(f"Ордер {order_id} был успешно отменен.")
#                                                     else:
#                                                         print(f"Не удалось отменить ордер {order_id}. Код ошибки: {cancel_response['retCode']}")
#                                             else:
#                                                 print(f"Не удалось получить открытые ордера. Код ошибки: {responseOpenOrders['retCode']}")
#                                             ##
#                                             last_signal = 0
                                            
#                                             break
#                                         elif order_info['list'][0]['orderStatus'] != "Filled":
#                                             ticker = session.get_tickers(category="spot", symbol="BTCUSDT")
#                                             price=float(ticker['result']['list'][0]['lastPrice'])
#                                             price=price
#                                             print(f"Новая цена {price}.")
#                                             order_response = place_order_sell(session, symbol="BTCUSDT", side="Sell", order_type="Limit", price=price, qty=0.1)
#                                             order_id = order_response['result']['orderId']

#                                             print(f"Ордер был отменен.Создали новый {order_id}")
#                                             continue
#                                     else:
#                                         print(f"Ошибка при проверке статуса ордера: {response['retMsg']}")
#                                         break  
#                                 except Exception as e:
#                                     print(f"Ошибка при проверке ордера: {e}")
                                time.sleep(0.1)  
                                
    except Exception as e:
        print(f"Произошла ошибка: {e}")

    time.sleep(0.5)

Сейчас время:
Mon Sep  9 20:15:00 2024
Смотрим на пред. свечу:
2024-09-09 17:14:00.465000
{'retCode': 0, 'retMsg': 'OK', 'result': {'symbol': 'BTCUSDT', 'category': 'linear', 'list': [['1725902040000', '56501.9', '56514.2', '56469.9', '56497', '266.619', '15061088.6459']]}, 'retExtInfo': {}, 'time': 1725902100134}


C:\Users\Waple\AppData\Local\Temp\ipykernel_788\2183328047.py:52: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  new_df['startTime'] = pd.to_datetime(new_df['startTime'], unit='ms')


,openPrice,highPrice,lowPrice,closePrice,volume,turnover,prev_close,tr1,tr2,tr3,tr,ATR14,AMA,MA21,MA63,EMA10,EMA30,EMA200,MOM10,MOM30,RSI10,RSI30,RSI200,%K10,%K30,%K200,%D10,%D30,%D200,MACD,Signal_Line
startTime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2024-08-30 07:13:00,59355.1,59364.1,59335.7,59364.0,56.374,3.345779e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-08-30 07:14:00,59364.0,59422.8,59364.0,59400.5,221.280,1.314173e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-08-30 07:15:00,59400.5,59400.5,59366.0,59367.8,49.948,2.965802e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-08-30 07:16:00,59367.8,59400.0,59364.0,59386.0,70.762,4.201520e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-08-30 07:17:00,59386.0,59407.8,59386.0,59406.5,36.583,2.172883e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-09 17:10:00,56696.1,56700.0,56517.3,56517.3,693.877,3.928578e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-09-09 17:11:00,56517.3,56616.0,56517.3,56548.2,279.527,1.580936e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-09-09 17:12:00,56548.2,56599.9,56548.2,56570.1,108.205,6.121096e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


None
предсказывает сигнаалл
[1]
Покупка BTC в Mon Sep  9 20:15:03 2024
Новая цена 56517.63.
Ордер был отменен.Создали новый 1771389669385440512
Новая цена 56512.81.
Ордер был отменен.Создали новый 1771389683889343744
Ордер 1771389683889343744 выполнен!
Сейчас время:
Mon Sep  9 20:16:00 2024
Смотрим на пред. свечу:
2024-09-09 17:15:00.153000
{'retCode': 0, 'retMsg': 'OK', 'result': {'symbol': 'BTCUSDT', 'category': 'linear', 'list': [['1725902100000', '56497', '56497', '56435.4', '56459.9', '268.775', '15176088.417']]}, 'retExtInfo': {}, 'time': 1725902159756}


C:\Users\Waple\AppData\Local\Temp\ipykernel_788\2183328047.py:52: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  new_df['startTime'] = pd.to_datetime(new_df['startTime'], unit='ms')


...

,openPrice,highPrice,lowPrice,closePrice,volume,turnover,prev_close,tr1,tr2,tr3,tr,ATR14,AMA,MA21,MA63,EMA10,EMA30,EMA200,MOM10,MOM30,RSI10,RSI30,RSI200,%K10,%K30,%K200,%D10,%D30,%D200,MACD,Signal_Line
startTime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2024-08-30 07:13:00,59355.1,59364.1,59335.7,59364.0,56.374,3.345779e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-08-30 07:14:00,59364.0,59422.8,59364.0,59400.5,221.280,1.314173e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-08-30 07:15:00,59400.5,59400.5,59366.0,59367.8,49.948,2.965802e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-08-30 07:16:00,59367.8,59400.0,59364.0,59386.0,70.762,4.201520e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-08-30 07:17:00,59386.0,59407.8,59386.0,59406.5,36.583,2.172883e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-09 18:36:00,56493.6,56527.2,56493.5,56509.9,62.317,3.521518e+06,56488.6,33.7,38.6,4.9,38.6,74.634146,56496.035837,56464.28,56453.876667,56462.325352,56465.953728,56155.579747,-26.6,13.8,57.118286,52.008271,54.308792,88.247283,65.080160,65.080160,47.327989,69.523279,85.463447,-2.721722,-5.859801
2024-09-09 18:37:00,56509.9,56509.9,56422.1,56426.3,60.609,3.421556e+06,56509.9,87.8,0.0,87.8,87.8,75.692683,56447.161682,56450.19,56459.990000,56455.775288,56463.395423,56158.273481,-140.9,183.4,45.652631,48.671332,53.737141,31.453804,24.746125,23.196393,48.078368,69.661141,85.448420,-5.360066,-5.759854
2024-09-09 18:38:00,56426.3,56426.3,56390.1,56405.3,62.050,3.500143e+06,56426.3,36.2,0.0,36.2,36.2,75.195122,56418.661106,56442.23,56454.946667,56446.597963,56459.647331,56160.731457,-79.6,-151.3,43.230569,47.873146,53.594721,17.187500,17.187500,12.675351,35.671262,49.279430,83.713074,-9.041275,-6.416138


None
предсказывает сигнаалл
[0]
Сейчас время:
Mon Sep  9 21:42:00 2024
Смотрим на пред. свечу:
2024-09-09 18:41:00.272000
{'retCode': 0, 'retMsg': 'OK', 'result': {'symbol': 'BTCUSDT', 'category': 'linear', 'list': [['1725907260000', '56390', '56470', '56385.3', '56461.3', '29.656', '1673108.7338']]}, 'retExtInfo': {}, 'time': 1725907319853}


C:\Users\Waple\AppData\Local\Temp\ipykernel_788\2183328047.py:52: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  new_df['startTime'] = pd.to_datetime(new_df['startTime'], unit='ms')


,openPrice,highPrice,lowPrice,closePrice,volume,turnover,prev_close,tr1,tr2,tr3,tr,ATR14,AMA,MA21,MA63,EMA10,EMA30,EMA200,MOM10,MOM30,RSI10,RSI30,RSI200,%K10,%K30,%K200,%D10,%D30,%D200,MACD,Signal_Line
startTime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2024-08-30 07:13:00,59355.1,59364.1,59335.7,59364.0,56.374,3.345779e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-08-30 07:14:00,59364.0,59422.8,59364.0,59400.5,221.280,1.314173e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-08-30 07:15:00,59400.5,59400.5,59366.0,59367.8,49.948,2.965802e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-08-30 07:16:00,59367.8,59400.0,59364.0,59386.0,70.762,4.201520e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-08-30 07:17:00,59386.0,59407.8,59386.0,59406.5,36.583,2.172883e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-09 18:37:00,56509.9,56509.9,56422.1,56426.3,60.609,3.421556e+06,56509.9,87.8,0.0,87.8,87.8,75.692683,56447.161682,56450.19,56459.990000,56455.775288,56463.395423,56158.273481,-140.9,183.4,45.652631,48.671332,53.737141,31.453804,24.746125,23.196393,48.078368,69.661141,85.448420,-5.360066,-5.759854
2024-09-09 18:38:00,56426.3,56426.3,56390.1,56405.3,62.050,3.500143e+06,56426.3,36.2,0.0,36.2,36.2,75.195122,56418.661106,56442.23,56454.946667,56446.597963,56459.647331,56160.731457,-79.6,-151.3,43.230569,47.873146,53.594721,17.187500,17.187500,12.675351,35.671262,49.279430,83.713074,-9.041275,-6.416138
2024-09-09 18:39:00,56404.8,56407.9,56355.0,56371.4,54.244,3.058064e+06,56405.3,52.9,2.6,50.3,52.9,75.082927,56386.383407,56433.35,56452.346667,56432.925606,56453.953955,56162.827661,-88.8,-78.0,39.474181,46.597022,53.365257,9.523810,9.523810,7.301870,17.152478,25.063982,80.822593,-14.526652,-8.038241


None
предсказывает сигнаалл
[0]


In [ ]:
from pybit.unified_trading import HTTP
session = HTTP(
    demo=True,
    api_key="",
    api_secret=""
)
history_order=session.get_order_history(
    category="spot",
    limit=1,
)
history_order

In [91]:
from pybit.unified_trading import HTTP

session = HTTP(
    demo=True,
    api_key="",
    api_secret=""
)

ticker = session.get_tickers(category="spot", symbol="BTCUSDT")
price=float(ticker['result']['list'][0]['lastPrice'])
side="Buy"
order_type="Limit"
qty=0.1
symbol = "BTCUSDT"

take_profit_price = price + 119
take_profit_price_limit = take_profit_price + 1

stopLoss_price=price-199
stopLoss_limit=stopLoss_price-1

order_response = session.place_order(
    category="spot",
    symbol=symbol,
    side=side,
    orderType=order_type,
    marketUnit="baseCoin",
    price=price,
    timeInForce="FOK",
    qty=qty,
    takeProfit=str(take_profit_price),
    tpLimitPrice=str(take_profit_price_limit),
    tpTriggerBy='LastPrice',
    tpOrderType="Limit",
    
    stopLoss=str(stopLoss_price),
    slLimitPrice=str(stopLoss_limit),
    slTriggerBy='LastPrice',
    slOrderType="Limit"
)

In [90]:
responseOpenOrders = session.get_open_orders(
    category="spot",
    symbol="BTCUSDT")

if responseOpenOrders['retCode'] == 0:
    tp_orders = []
    for order in responseOpenOrders['result']['list']:
        print(order)

In [92]:
history_order=session.get_order_history(
category="spot",
limit=1)
history_order

{'retCode': 0,
 'retMsg': 'OK',
 'result': {'nextPageCursor': '1771264882659036419%3A1725887228268%2C1771264882659036419%3A1725887228268',
  'category': 'spot',
  'list': [{'symbol': 'BTCUSDT',
    'orderType': 'Limit',
    'orderLinkId': '1771264882659036420',
    'slLimitPrice': '0',
    'orderId': '1771264882659036419',
    'cancelType': 'UNKNOWN',
    'avgPrice': '55289.68',
    'stopOrderType': 'BidirectionalTpslOrder',
    'lastPriceOnCreated': '',
    'orderStatus': 'Filled',
    'takeProfit': '0',
    'cumExecValue': '5523.43903200',
    'smpType': 'None',
    'triggerDirection': 0,
    'blockTradeId': '',
    'rejectReason': 'EC_NoError',
    'isLeverage': '0',
    'price': '55288.85',
    'orderIv': '',
    'createdTime': '1725887228268',
    'tpTriggerBy': '',
    'positionIdx': 0,
    'trailingPercentage': '0',
    'timeInForce': 'GTC',
    'leavesValue': '0.00000000',
    'basePrice': '55294.84',
    'updatedTime': '1725887230944',
    'side': 'Sell',
    'smpGroup': 0,
  